In [1]:
# Bring in (new) PNC cohort

import pickle
from pathlib import Path
import numpy as np

newdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP'
newdemo = pickle.load(open(f'{newdir}/demographics.pkl', 'rb'))

newfc = dict()

for sub in newdemo['Age_cal']:
    for task in ['unk']:
        f = f'{newdir}/fc/{sub}_task-{task}_fc.npy'
        if not Path(f).exists():
            continue
        p = np.load(f)
        newfc[f'{sub}'] = p

print(len(newfc))

1244


In [2]:
# Angle estimate

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

pi = 3.14
pi2 = 2*pi

def tonp(t):
    return t.detach().cpu().numpy()

class AngleBasis(nn.Module):
    def __init__(self, mixn):
        super(AngleBasis, self).__init__()
        self.mixn = mixn
        self.thetas = nn.Parameter((pi*torch.rand(self.mixn,264)+pi/2).float().cuda())
        self.jitter = nn.Parameter(torch.ones(self.mixn,264).float().cuda())
        
    def project(self):
        with torch.no_grad():
            self.jitter[self.jitter < 0] = 0
            self.jitter[self.jitter > 1] = 1
        
    def phases(self):
        t0 = self.thetas.unsqueeze(2)
        t1 = self.thetas.unsqueeze(1)
        return t0-t1
    
    def jit(self):
        j0 = self.jitter.unsqueeze(2)
        j1 = self.jitter.unsqueeze(1)
        return j0*j1
    
    def ps(self, jitter=False):
        t = self.phases()
        p = torch.cos(t)
        if jitter:
            j = self.jit()
            p = j*p
        return p
    
    def dump(self):
        return dict(mixn=self.mixn, thetas=tonp(self.thetas), jitter=tonp(self.jitter))
    
    def psum(self):
        return torch.mean(self.ps(), axis=0)
    
    def pvec(self):
        a,b = np.triu_indices(264,1)
        p = self.psum()
        return p[a,b]
    
print('Complete')

Complete


In [9]:
nepochs = 1000
pperiod = 500

def fromnp(x):
    return torch.from_numpy(x).float().cuda()

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

bases = dict()
i = 0

for subtask in newfc:
    x = fromnp(newfc[subtask])

    min_loss = 10
    sav = None

    for _ in range(2):
        basis = AngleBasis(1)
        optim = torch.optim.Adam(basis.parameters(), lr=1e-2, weight_decay=0)

        for e in range(nepochs):
            optim.zero_grad()
            xhat = basis.pvec()
            loss = rmse(xhat, x)
            loss.backward()
            optim.step()
            basis.project()
            if e == nepochs-1 or e % pperiod == 1:
                print(f'{e} {float(loss)}')

        if loss < min_loss:
            print('Saving')
            min_loss = float(loss)
            sav = basis
    
    bases[subtask] = sav.dump()
    i += 1
    
    print(f'Finished {i} {subtask}')
        
print('Complete')

1 0.6455332636833191
501 0.48152226209640503
999 0.4804592728614807
Saving
1 0.6597307324409485
501 0.4839077293872833
999 0.4816971719264984
Finished 1 Baltimore.S0015SRH1
1 0.7023962140083313
501 0.5982518792152405
999 0.5982514023780823
Saving
1 0.7051042914390564
501 0.5982674360275269
999 0.5982514023780823
Finished 2 Baltimore.S0021FHS1
1 0.6047566533088684
501 0.5193042159080505
999 0.5109172463417053
Saving
1 0.607479453086853
501 0.5127921104431152
999 0.5085237622261047
Saving
Finished 3 Baltimore.S0172LAC1
1 0.672198474407196
501 0.5255579352378845
999 0.525547444820404
Saving
1 0.6795860528945923
501 0.5256850719451904
999 0.5256626605987549
Finished 4 Baltimore.S0186SQN1
1 0.6598041653633118
501 0.5005592107772827
999 0.49526700377464294
Saving
1 0.6541134119033813
501 0.49745750427246094
999 0.4952881634235382
Finished 5 Baltimore.S0237KSC1
1 0.6339977979660034
501 0.49537745118141174
999 0.4779282510280609
Saving
1 0.6173141598701477
501 0.4594012200832367
999 0.45860111

501 0.44824156165122986
999 0.44796809554100037
Saving
1 0.6327335834503174
501 0.4473191499710083
999 0.4463164806365967
Saving
Finished 48 Baltimore.S2004XSP1
1 0.650077760219574
501 0.5901570320129395
999 0.585125207901001
Saving
1 0.6536972522735596
501 0.5837600827217102
999 0.5824013352394104
Saving
Finished 49 Baltimore.S2082VSV1
1 0.6547681093215942
501 0.42046216130256653
999 0.4204121530056
Saving
1 0.6488221883773804
501 0.4520750045776367
999 0.4188711643218994
Saving
Finished 50 Baltimore.S2129GQR1
1 0.6217215657234192
501 0.3945964574813843
999 0.3902454078197479
Saving
1 0.6393911838531494
501 0.3975081443786621
999 0.3921707570552826
Finished 51 Baltimore.S2144JPC1
1 0.7013147473335266
501 0.6190299987792969
999 0.616153359413147
Saving
1 0.7038301229476929
501 0.6250394582748413
999 0.6190097332000732
Finished 52 Baltimore.S2180THQ1
1 0.6736065745353699
501 0.6168922185897827
999 0.6136540770530701
Saving
1 0.6732626557350159
501 0.6177549958229065
999 0.61355513334274

501 0.43979403376579285
999 0.43969979882240295
Saving
Finished 94 Baltimore.S3363WCM1
1 0.6483819484710693
501 0.505628228187561
999 0.5052396655082703
Saving
1 0.6475080251693726
501 0.5068167448043823
999 0.5063663721084595
Finished 95 Baltimore.S3367DPS1
1 0.6667237281799316
501 0.5826573967933655
999 0.5821044445037842
Saving
1 0.6727681159973145
501 0.5648414492607117
999 0.562019944190979
Saving
Finished 96 Baltimore.S3379QVT1
1 0.6844926476478577
501 0.6172045469284058
999 0.6157060265541077
Saving
1 0.6797981858253479
501 0.6154617667198181
999 0.6154614090919495
Saving
Finished 97 Baltimore.S3425IWD1
1 0.5933808088302612
501 0.44284752011299133
999 0.41352343559265137
Saving
1 0.596325695514679
501 0.4600687026977539
999 0.43549397587776184
Finished 98 Baltimore.S3481WNG1
1 0.6241580247879028
501 0.47206881642341614
999 0.4711852967739105
Saving
1 0.6298161149024963
501 0.47524595260620117
999 0.47429516911506653
Finished 99 Baltimore.S3546AYK1
1 0.6318961381912231
501 0.5465

501 0.5421785116195679
999 0.53033447265625
Saving
Finished 140 Baltimore.S4862CET1
1 0.6061582565307617
501 0.36860552430152893
999 0.35898324847221375
Saving
1 0.6481761932373047
501 0.3869982659816742
999 0.3619641661643982
Finished 141 Baltimore.S4900GHI1
1 0.6137741208076477
501 0.4641961455345154
999 0.46267595887184143
Saving
1 0.6190935373306274
501 0.46522319316864014
999 0.46314823627471924
Finished 142 Baltimore.S4915NCC1
1 0.643459677696228
501 0.5172926783561707
999 0.5143581032752991
Saving
1 0.6529086232185364
501 0.5401608943939209
999 0.537323534488678
Finished 143 Baltimore.S4929VIB1
1 0.6276983618736267
501 0.5198647379875183
999 0.5159915685653687
Saving
1 0.6403453350067139
501 0.4928649067878723
999 0.48971027135849
Saving
Finished 144 Baltimore.S5013WSJ1
1 0.6377302408218384
501 0.4876103103160858
999 0.4867004156112671
Saving
1 0.6120229959487915
501 0.49442145228385925
999 0.49253201484680176
Finished 145 Baltimore.S5044MKQ1
1 0.6774295568466187
501 0.626207590

501 0.5699819922447205
999 0.5641425848007202
Saving
1 0.6618601083755493
501 0.5683966279029846
999 0.564619243144989
Finished 187 Baltimore.S6520RRB1
1 0.6373487710952759
501 0.5326324105262756
999 0.5322825312614441
Saving
1 0.6636381149291992
501 0.549018144607544
999 0.5485608577728271
Finished 188 Baltimore.S6536EIF1
1 0.6105303168296814
501 0.4514331817626953
999 0.44864797592163086
Saving
1 0.5920674800872803
501 0.4455292522907257
999 0.44142740964889526
Saving
Finished 189 Baltimore.S6547ONC1
1 0.6603127717971802
501 0.5731727480888367
999 0.5656708478927612
Saving
1 0.6617323160171509
501 0.5779139399528503
999 0.5694513320922852
Finished 190 Baltimore.S6576UVX1
1 0.6732355952262878
501 0.29872849583625793
999 0.2919274568557739
Saving
1 0.6079146862030029
501 0.2925570607185364
999 0.2920375168323517
Finished 191 Baltimore.S6614YFO1
1 0.6267452239990234
501 0.5019161701202393
999 0.5002622008323669
Saving
1 0.6283536553382874
501 0.5140677094459534
999 0.49941158294677734
S

...

999 0.4484073519706726
Finished 1026 Hartford.S3939JFS1
1 0.6016170382499695
501 0.48408105969429016
999 0.48228734731674194
Saving
1 0.6086061596870422
501 0.4885980486869812
999 0.48579832911491394
Finished 1027 Hartford.S3947TCF1
1 0.6705598831176758
501 0.5734100937843323
999 0.5616194605827332
Saving
1 0.6674013137817383
501 0.5681315660476685
999 0.5603879690170288
Saving
Finished 1028 Hartford.S3984MKD2
1 0.6485363245010376
501 0.5278059840202332
999 0.5261099934577942
Saving
1 0.6742441058158875
501 0.5307301878929138
999 0.5291678309440613
Finished 1029 Hartford.S3984WUL1
1 0.6432819366455078
501 0.5044902563095093
999 0.5036117434501648
Saving
1 0.6364469528198242
501 0.5018563866615295
999 0.5017631649971008
Saving
Finished 1030 Hartford.S3986HWE1
1 0.6294708251953125
501 0.4652942717075348
999 0.46418625116348267
Saving
1 0.6351044178009033
501 0.4679422676563263
999 0.46782585978507996
Finished 1031 Hartford.S4048LKF1
1 0.670054018497467
501 0.5610551238059998
999 0.561055

501 0.501028835773468
999 0.49593621492385864
Saving
1 0.6204975843429565
501 0.4955209195613861
999 0.4954126477241516
Saving
Finished 1073 Hartford.S5200EFE1
1 0.6701058149337769
501 0.6038662791252136
999 0.5978944897651672
Saving
1 0.6688485145568848
501 0.6035320162773132
999 0.6005970239639282
Finished 1074 Hartford.S5249IVW1
1 0.6652781367301941
501 0.5639225840568542
999 0.5630247592926025
Saving
1 0.666118860244751
501 0.5642152428627014
999 0.5636410117149353
Finished 1075 Hartford.S5273ALO1
1 0.6484104990959167
501 0.5116945505142212
999 0.4894077777862549
Saving
1 0.6422114372253418
501 0.5114966630935669
999 0.5062375664710999
Finished 1076 Hartford.S5300CRN1
1 0.6104022264480591
501 0.5342989563941956
999 0.5329511761665344
Saving
1 0.6237237453460693
501 0.5309759974479675
999 0.5282992124557495
Saving
Finished 1077 Hartford.S5331PHP1
1 0.6343680024147034
501 0.41341301798820496
999 0.4133921265602112
Saving
1 0.6356951594352722
501 0.41328802704811096
999 0.409906208515

501 0.5704588294029236
999 0.5704588294029236
Finished 1119 Hartford.S6414SXU1
1 0.5991803407669067
501 0.47298747301101685
999 0.4653984606266022
Saving
1 0.6271262764930725
501 0.48788580298423767
999 0.46508824825286865
Saving
Finished 1120 Hartford.S6428VMW2
1 0.6348157525062561
501 0.519412636756897
999 0.5175127983093262
Saving
1 0.6464921236038208
501 0.5174596309661865
999 0.517459511756897
Saving
Finished 1121 Hartford.S6447DGM1
1 0.6262408494949341
501 0.533549427986145
999 0.5315018892288208
Saving
1 0.6348387598991394
501 0.5264438986778259
999 0.5200921297073364
Saving
Finished 1122 Hartford.S6451JFH1
1 0.5987472534179688
501 0.5003909468650818
999 0.4998149871826172
Saving
1 0.6114036440849304
501 0.4998464286327362
999 0.4993354082107544
Saving
Finished 1123 Hartford.S6480AET3
1 0.6642361283302307
501 0.5515279173851013
999 0.5503087639808655
Saving
1 0.6602210998535156
501 0.5498822927474976
999 0.5482626557350159
Saving
Finished 1124 Hartford.S6486OAH1
1 0.616474747657

999 0.5158065557479858
Finished 1165 Hartford.S8020JKG2
1 0.6668537259101868
501 0.47796884179115295
999 0.4775150716304779
Saving
1 0.6862184405326843
501 0.5358937978744507
999 0.4775150716304779
Finished 1166 Hartford.S8032TUN2
1 0.6609039902687073
501 0.539838969707489
999 0.5379287600517273
Saving
1 0.6503641605377197
501 0.5369484424591064
999 0.5368477702140808
Saving
Finished 1167 Hartford.S8072RYL3
1 0.6456458568572998
501 0.5610186457633972
999 0.5610094666481018
Saving
1 0.6587722897529602
501 0.5642827749252319
999 0.5614458918571472
Finished 1168 Hartford.S8095VIP1
1 0.6186465620994568
501 0.515597403049469
999 0.49630293250083923
Saving
1 0.6204045414924622
501 0.5175348520278931
999 0.49275457859039307
Saving
Finished 1169 Hartford.S8123VLA1
1 0.6653571724891663
501 0.5769583582878113
999 0.5754231214523315
Saving
1 0.6612993478775024
501 0.5768898725509644
999 0.5767976641654968
Finished 1170 Hartford.S8156OYE2
1 0.6725437641143799
501 0.5382643938064575
999 0.536491513

501 0.5356290340423584
999 0.5346243381500244
Saving
1 0.6479105949401855
501 0.536561906337738
999 0.5365594625473022
Finished 1212 Hartford.S9335EXE3
1 0.5952787399291992
501 0.4862883687019348
999 0.47739261388778687
Saving
1 0.6053304672241211
501 0.49049657583236694
999 0.48003333806991577
Finished 1213 Hartford.S9358LVU1
1 0.6710773706436157
501 0.5642845630645752
999 0.5614485740661621
Saving
1 0.6730080246925354
501 0.5918289422988892
999 0.567486047744751
Finished 1214 Hartford.S9403UJQ1
1 0.6148290634155273
501 0.4674789309501648
999 0.46304768323898315
Saving
1 0.6081780195236206
501 0.46413537859916687
999 0.4584878087043762
Saving
Finished 1215 Hartford.S9438IKO1
1 0.6121575832366943
501 0.4557887315750122
999 0.43485310673713684
Saving
1 0.6174095273017883
501 0.44577810168266296
999 0.42193275690078735
Saving
Finished 1216 Hartford.S9445CEO1
1 0.6695922613143921
501 0.5889962911605835
999 0.56038898229599
Saving
1 0.6566694378852844
501 0.5617622137069702
999 0.561151087

In [10]:
import pickle

pickle.dump(bases, open('/home/anton/Documents/Tulane/Research/BSNIP/AngleBasisNoJit1.pkl', 'wb'))

print('Complete')

Complete
